![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

### Principle Component Analysis
Briefly stated, collinearity is the state of two independent variables being highly correlated. Unfortunately, collinearity can cause trouble when using regression models. One of the fundamental assumptions of Ordinary Least Squares regression is that the variables can't have a linear relationship. This problem is especially tricky when dealing with multiple variables in regression models where multicollinearity occurs. The presence of multicollinearity is a problem because the linear regression model cannot distinguish between the co-lineated variables. It may cause some variables to appear to be insignificant in the relationship when they really are - in other words, their "weight" is transferred to another, correlated value.

Principal Component Analysis (PCA) a way of mapping the existing dataset into a new "space", where the dimensions of the new data are linearly-independent, orthogonal vectors. PCA eliminates the problem of multicollinearity. In addition to this, PCA gives us a way of identifying the dimensions of the data that contribute most to the variance of the data, meaning we can also use PCA to reduce the number of input variables in our regression model. We can use the PCA-transformed data to build a regression model, make predictions, and then map those predictions back into the original data "space" so that we have applicable predictions.

In [1]:
import numpy as np
import statsmodels.api as sm
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression, HuberRegressor, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV

# Import the Liquid ETF Universe helper methods
from QuantConnect.Data.UniverseSelection import *
from QuantConnect.Data.Custom.USTreasury import *

# Initialize QuantBook and the US Treasuries ETFs
qb = QuantBook()
yieldCurve = qb.AddData(USTreasuryYieldCurveRate, "USTYCR", Resolution.Daily).Symbol

In [2]:
# Get history
history = qb.History(yieldCurve, 100, Resolution.Daily)
# Get prices and returns
bonds = history.loc[yieldCurve].pct_change().ffill().fillna(value = 0).replace([np.inf, -np.inf], np.nan).dropna()
bonds.head()

,fiveyear,onemonth,oneyear,sevenyear,sixmonth,tenyear,thirtyyear,threemonth,threeyear,twentyyear,twomonth,twoyear
time,,,,,,,,,,,,
2020-01-28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-29,-0.040816,-0.006536,-0.013072,-0.032051,-0.006329,-0.030303,-0.023810,-0.006369,-0.041379,-0.030769,-0.012739,-0.020690
2020-01-30,-0.014184,0.046053,-0.019868,-0.013245,0.000000,-0.018750,-0.004878,0.006410,-0.014388,-0.005291,0.019355,-0.007042
2020-01-31,-0.050360,-0.018868,-0.020270,-0.046980,-0.019108,-0.038217,-0.024510,-0.012739,-0.051095,-0.026596,-0.006329,-0.056738
2020-02-03,0.022727,0.000000,0.006897,0.021127,0.012987,0.019868,0.010050,0.012903,0.030769,0.005464,0.000000,0.022556


In [3]:
# Structure data for modeling - train for building the models, testing to make predictions with
training = bonds.iloc[:len(bonds)-1].copy()
testing = bonds.iloc[len(bonds)-1:].copy()

In [4]:
# The implementation of the function in RegressionFunction.py but with some added plotting features
def FitRegressionModel(pca, model, training, testing, alpha = False):    
    estimators = []
    Y_pred = []
    Y_actual = []
    X_training_proj = pca.transform(training)

    # Iterate over all principle components:
    for i in range(pca.n_components_):

        # Here, we lag X compared to Y. X will be one time period behind Y
        X = X_training_proj[:-1, i]
        Y = X_training_proj[1:, i]
        
        Y_actual.append(Y)
        X = sm.add_constant(X)
        
        if alpha:
            test_range = np.arange(5)
            param_grid = {"alpha": test_range}
            grid_search = GridSearchCV(model,param_grid)
            grid_search.fit(X, Y)
            best_params = grid_search.best_params_
        
        est = model.fit(X, Y)
        estimators.append(est)
        Y_pred.append(model.predict(X))
        print("Estimator {}: R2 = {:.3f}\n".format(i, model.score(X,Y)))
    
    Y_pred = np.array(Y_pred).transpose()
    Y_actual = np.array(Y_actual).transpose()
    
    Y_actual_original_space = pca.inverse_transform(Y_actual)
    Y_pred_original_space = pca.inverse_transform(Y_pred)

    # Compute sum of squared error:
    train_sse = np.sum((Y_pred_original_space - Y_actual_original_space)** 2)
    print(f'Sum of squared error: {train_sse}\n')
    testing_proj = pca.transform(testing)
    
    testing_prediction = []
    for i in range(pca.n_components_):
        # Create a data row - remember our estimators have a constant, so we need that
        row = [1, testing_proj[:,i]]
        print(row)
        row = np.reshape(row, (1, 2))
        # Predict this row
        p = model.predict(row)
        # Transofrm the (1, 1) result into just a (1,), and append to our predictions

        # Potential error here:
        testing_prediction.append(p[0])  # If this errors, try p[0][0]
        
    predictions = pca.inverse_transform(testing_prediction)
    pred_sse = np.sum((predictions - testing.values)** 2)
    actual_pred = {'Predicted':predictions, 'Actual':[x[0] for x in testing.transpose().values]}
    actual_pred = pd.DataFrame(actual_pred, index = testing.columns)
    print(actual_pred)
    print(f'\nPrediction sum of squared error: {pred_sse}\n')
    return actual_pred['Predicted']

Instead of telling the PCA model how many components we wanted to keep, we decided instead to have it return the number of components it took to explain 95% of the variance of the data, which in the case of treasury yield changes was 4.

In [5]:
# Initialize the PCA model
pca = PCA(n_components=0.95)  # Forces it to explain >99% of variance
# Fit the PCA model
pca.fit(training)
print(f'PCA Explained Variance: {pca.explained_variance_ratio_}\n')
print(f'PCA No. Components: {pca.n_components_}\n')

PCA Explained Variance: [0.76330253 0.08974696 0.0675421  0.04663491]

PCA No. Components: 4



We fit two different regression models -- Linean and Random Forest

In [6]:
# Initialize a standard OLS model
model = LinearRegression()
results = FitRegressionModel(pca, model, training, testing)

Estimator 0: R2 = 0.008

Estimator 1: R2 = 0.043

Estimator 2: R2 = 0.023

Estimator 3: R2 = 0.000

Sum of squared error: 67.20596541646557

[1, array([-0.10059391])]
[1, array([-0.15233152])]
[1, array([-0.01927581])]
[1, array([-0.0198384])]
            Predicted    Actual
fiveyear    -0.009274  0.027778
onemonth     0.008572 -0.111111
oneyear     -0.015760  0.000000
sevenyear   -0.007291  0.075472
sixmonth     0.049502  0.066667
tenyear     -0.005246  0.090909
thirtyyear  -0.002515  0.068182
threemonth  -0.001092 -0.076923
threeyear   -0.014679  0.000000
twentyyear  -0.003784  0.084112
twomonth    -0.022414 -0.090909
twoyear     -0.019232 -0.105263

Prediction sum of squared error: 0.06311788675635607



In [7]:
# Initialize Randome Forest Regression model
model = RandomForestRegressor(random_state=0, n_estimators = 100)
results = FitRegressionModel(pca, model, training, testing)

Estimator 0: R2 = 0.814

Estimator 1: R2 = 0.835

Estimator 2: R2 = 0.822

Estimator 3: R2 = 0.816

Sum of squared error: 12.488616430427138

[1, array([-0.10059391])]
[1, array([-0.15233152])]
[1, array([-0.01927581])]
[1, array([-0.0198384])]
            Predicted    Actual
fiveyear     0.075139  0.027778
onemonth    -0.118144 -0.111111
oneyear      0.050716  0.000000
sevenyear    0.058816  0.075472
sixmonth     0.226376  0.066667
tenyear      0.065140  0.090909
thirtyyear   0.042342  0.068182
threemonth  -0.016886 -0.076923
threeyear    0.069739  0.000000
twentyyear   0.047773  0.084112
twomonth    -0.138771 -0.090909
twoyear      0.053890 -0.105263

Prediction sum of squared error: 0.06938996854271938



Random Forest Regression has the highest R-squared values and the prediction SSE is approximately the same as Linear Regression, so RFR is the model we would want to use in an algorithm.